In [49]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import os
import gzip
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from matplotlib.colors import LogNorm
from scipy.stats import multivariate_normal
import pandas as pd

load minst copied from the fashion-mnist datasets's utils. This method is used to load the dataset

In [ ]:
def load_mnist(path, kind='train'):
    
    ''' from https://github.com/zalandoresearch/fashion-mnist'''

    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

Principle components analysis or PCA, is a technique for reducing the dimensionality of datasets. The aim is to increase a databases interpretability while minimizing information loss. This is achieved by creating new uncorrelated variables that successively maximize variance. in the cell below, the dataset is loaded and then standardized by dividing them by 225 since they are all pixel values. Furthermore, their mean is calculated, and the data is normalized to make the PCA appliance easier. The first 25 images are shown with matching labels in the block below. Method 'conv_2d' splits the 1d input of 784 pixels into 2d with the shape (28,28) to make the plotting possible.

In [ ]:
fashion_X_train, train_y_labels = load_mnist('C:/Uni/Year3/MLCW//fashion-mnist/data/fashion', kind='train')
fashion_X_test, test_y_labels = load_mnist('C:/Uni/Year3/MLCW/fashion-mnist/data/fashion', kind='t10k')

class_names = ['t-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'bag', 'ankle boot']

#standardization: devided by 255
x_train_standard = fashion_X_train/255
x_test_standard = fashion_X_test/255

#normalizing by finding mean and calculating distance from mean
train_mean = np.mean(x_train_standard)
test_mean = np.mean(x_test_standard)

train_normal = x_train_standard - train_mean
test_normal = x_test_standard - test_mean


def conv_2d(matrix):
    for x in matrix:
        matrix = np.reshape(matrix, (28,28))
    return matrix

plt.figure(figsize=(7,7))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    train_images = conv_2d(fashion_X_train[i])
    plt.imshow(train_images, cmap=plt.cm.binary)
    plt.xlabel(class_names[train_y_labels[i]])
plt.show()

In the following block, the dimensionality of the data is reduced to only two components from 784. PCA is performed here to understand whether this reduction significantly impacts our information retention or not. a good reduction will aim to keep most of the information while discarding as many dimensions as possible. Here, we learn some quantities about these data namely different components and cumulative variance of the components. These represent the principal components of the data. in the code cell below evector of >=0.8 is taken from the cumulative variance as we want to retain 80+% of our datapoints as it is the ideal number.

In [ ]:
pca = PCA()
fit = pca.fit_transform(train_normal)

cumilative = np.cumsum(pca.explained_variance_)

print(f"first 5 PCs: {pca.explained_variance_[:10]}")


ex_var_ratio = pca.explained_variance_ratio_
cum_var_ratio = np.cumsum(ex_var_ratio)
first_evec_90 = np.where(cum_var_ratio >= 0.8)[0][0] #evector with cumulative variance ratio of >=0.8

# Plot cumulative explained variance ratio
principle_components = list(range(1, len(ex_var_ratio)+1))
plt.plot(principle_components, np.cumsum(ex_var_ratio), color= 'black')
plt.annotate("PC = "+str(first_evec_90+1)+", CVR = 80%", (80, 0.9)).set_fontsize(15)
plt.xlabel('Kth Principal Component')
plt.ylabel('Cumulative Variance Ratio')
plt.show()

In the cell above, the PCA was applied to all data and the pca object of the first 10 principles were returned. With this analysis we intend to keep higher variances. The output tells us that the cumalitive variance decreases rapidly. Furthermore, to understand how many dimension would be ideal to retain, Cumulative Variance Ratio per Principal Component is plotted. This better visualizes the pca of whole data. It can be observed that the first "24" principle components can explain 80% of the variance in this dataset. so reducing the datasets to 24 dimensions is ideal.

in the cell below we reduce the dimensionality of our data to 2 components. the data is then visualized as we plot the first component against the second component. Cumulative variance of the two components are also printed.

In [ ]:
pca_2 = PCA(n_components=2)
fit_2d = pca_2.fit_transform(train_normal)
cum_var = pca_2.explained_variance_#cumilitive variance
cumalitive_var = np.cumsum(cum_var)
print(cumalitive_var)


plt.figure(figsize=(10,10))
scatter = plt.scatter(fit_2d.T[0], fit_2d.T[1], c=train_y_labels)
clb = plt.colorbar(scatter) #colour coding
clb.ax.set_title('Class') #colour coding
plt.xlabel("1st principle comp", fontsize=12)
plt.ylabel("2nd principle comp", fontsize=12)
plt.title('2D pca')

The first principle component is "19.80980567" and the second principle component is "31.92201614" clearly showing the gap between the two components are high. This tells us that reducing to 2d from 780+ dimensions is not a good idea since all the classes keep clustering into each other and there is no clear separation between the classes and most of the datapoint are lost and we are underfitting our data. We can furthur prove that this reduction is not fitting with plotting our images with this reduction

In [ ]:
# Plot the grid of images
fig, axes = plt.subplots(2,2, figsize = (10,5), sharex=True, sharey = True, constrained_layout = True )

# First 2 principal components
f_2_PCS = [pca.components_[i].reshape(28,28) for i in range(0,2)]

for pc in range(0,2):
    axes[0,pc].imshow(f_2_PCS[0], aspect='auto', cmap = "gray_r")
    axes[1,pc].imshow(f_2_PCS[pc], aspect='auto', cmap = "gray_r")
plt.show()

It is demonstrated that the images come out extremly blurry and do not really represeant all of the shapes we will see within our dataset. Moreover, these images show that most variance in the data is found between the pixel values of a dark shirt and the pixel values of a white shoe.

in the code block below a simple K-means clustering is being employed. Each data is being assigned a cluster and cluster centres are calculated and marked inside each cluster. This is an unsupervised task with the task to separate points inside a dataset into different clusters such that elements within each cluster are similar. GMM is an example of soft clustering while K-mean is an example of hard clustering

In [ ]:
kmeans = KMeans(n_clusters=10)
label = kmeans.fit_predict(fit_2d)
centroid = kmeans.cluster_centers_
unique = np.unique(label)

plt.figure(figsize=(10,10))
for i in unique:
    plt.scatter(fit_2d[label == i , 0] , fit_2d[label == i , 1] , label = i)
    plt.scatter(centroid[:,0], centroid[:,1], s = 80, color = 'b')
plt.show()

GMM

Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. The probability of where each datapoints belong to and plot them below, since GMM is a probabilistic model we use them to fit and find clusters for a number of our data points by using the fit_predict() method and scatter the datapoints with colours corresponding with their assigned clusters. Furthermore, the cluster centroids are represented by a grey circle within the scatters. z-score for this model is "-5.041879030344901" with some minor overlapping happening between the classes. Keeping in mind only 300 sample data were used for optimization. This score reveals that these datapoints are about 5 standard deviations away from the mean. Data's standard deviation is "4.017053219615373" while the mean is "0.08295799598790543". Since there is a large gap, it suggests that our data are scattered and not close to each other.

In [ ]:
colours = ['red', 'green', 'purple', 'yellow', 'blue', 'cyan', 'black', 'grey', 'orange', 'brown']

gmm = GaussianMixture(n_components=10, covariance_type='full', random_state=0)
x = fit_2d[:300,:300]
mea = gmm.fit(x).means_
means = np.mean(mea)
labels = gmm.fit_predict(x)
score = gmm.score(x)
std = np.std(x)
print('std', std)
print('mean', means)
print('score', score)

centers = np.empty(shape=(gmm.n_components, x.shape[1]))
mean_centre = gmm.means_
cov_centre = gmm.covariances_
plt.figure(figsize=(10,10))
for i in range(gmm.n_components):
    density = multivariate_normal(mean_centre[i], cov_centre[i]).logpdf(x)
    centers[i, :] = x[np.argmax(density)]
plt.scatter(centers[:, 0], centers[:, 1], s=500, c='grey', marker='X')
pred = plt.scatter(x[:, 0], x[:, 1], c=labels, cmap='viridis', s=20) #predictions
clb = plt.colorbar(pred) #colour coding
clb.ax.set_title('Class labels') #colour coding
plt.show()

code cell below plots and compares our actual datapoints against the prediction from the Gaussian mixture model